# CSCI236 assignment 1 Task1 
* Name: Nguyen Gia Khanh
* UOWID: 7311217


### Import libraries

In [32]:
import pandas as pd
import numpy as np

### 1. Read csv file as pandas dataframe

In [33]:
df1 = pd.read_csv("customer_churn_dataset-training-master.csv")
df2 = pd.read_csv("customer_churn_dataset-testing-master.csv")
df = pd.concat([df1, df2], axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 505207 entries, 0 to 64373
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         505206 non-null  float64
 1   Age                505206 non-null  float64
 2   Gender             505206 non-null  object 
 3   Tenure             505206 non-null  float64
 4   Usage Frequency    505206 non-null  float64
 5   Support Calls      505206 non-null  float64
 6   Payment Delay      505206 non-null  float64
 7   Subscription Type  505206 non-null  object 
 8   Contract Length    505206 non-null  object 
 9   Total Spend        505206 non-null  float64
 10  Last Interaction   505206 non-null  float64
 11  Churn              505206 non-null  float64
dtypes: float64(9), object(3)
memory usage: 50.1+ MB


### 2. Identify missing values

In [34]:
# get general info about test dataset
df.isnull().sum()

CustomerID           1
Age                  1
Gender               1
Tenure               1
Usage Frequency      1
Support Calls        1
Payment Delay        1
Subscription Type    1
Contract Length      1
Total Spend          1
Last Interaction     1
Churn                1
dtype: int64

Check for missing value in train dataset

In [35]:
missing_value = df[df.isnull().any(axis=1)]
missing_value

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
199295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In train dataset, row 199295 contains missing values <br>
We can safely remove the whole data point because: <br>
- This is the only record that has missing value (<5% dataset's size)
- It missing all of its attribute


In [36]:
df = df.dropna()

### 3. Using z-score normalization for column "Last Interaction"

In [37]:
train_mean_li = df["Last Interaction"].mean()
train_std_li = df["Last Interaction"].std()
df["LI z-score"] = (df["Last Interaction"] - train_mean_li)/train_std_li
# Swap the values and column names
df['Churn'], df['LI z-score']  = df['LI z-score'], df['Churn']
df.rename(columns={'Churn': 'LI z-score', 'LI z-score': 'Churn'}, 
                    inplace=True)
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,LI z-score,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,0.277572,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,-1.000267,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,-1.348768,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.671578,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,0.626073,1.0


Mean of normalized data

In [38]:
round(df["LI z-score"].mean())

0

Variance of normalized data

In [39]:
df["LI z-score"].var()

0.9999999999999998

### 4. Create five bins for the attribute “Total Spend” such that the bins contain (approximately) equivalent numbers of records

In [40]:
bin_labels = ['Bin1', 'Bin2', 'Bin3', 'Bin4', 'Bin5']
df['Total Spend'] = pd.cut(df['Total Spend'], bins=5, labels=bin_labels)

### 5.  Apply one-hot-encoding to the attribute “Contract Length”


In [41]:
ohe = pd.get_dummies(df["Contract Length"]).astype(int)
df = pd.concat([df, ohe], axis=1)
# Specify the desired column order
desired_order = [
    'CustomerID', 'Age', 'Gender', 'Tenure', 'Usage Frequency', 'Support Calls',
    'Payment Delay', 'Subscription Type', 'Contract Length', 'Total Spend',
    'Last Interaction', 'LI z-score', 'Annual', 'Monthly', 'Quarterly','Churn'
]

# Reorder the columns in the DataFrame
df = df[desired_order]
df

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,LI z-score,Annual,Monthly,Quarterly,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,Bin5,17.0,0.277572,1,0,0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,Bin3,6.0,-1.000267,0,1,0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,Bin1,3.0,-1.348768,0,0,1,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,Bin2,29.0,1.671578,0,1,0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,Bin3,20.0,0.626073,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64369,64370.0,45.0,Female,33.0,12.0,6.0,21.0,Basic,Quarterly,Bin5,14.0,-0.070929,0,0,1,1.0
64370,64371.0,37.0,Male,6.0,1.0,5.0,22.0,Standard,Annual,Bin5,9.0,-0.651765,1,0,0,1.0
64371,64372.0,25.0,Male,39.0,14.0,8.0,30.0,Premium,Monthly,Bin2,20.0,0.626073,0,1,0,1.0
64372,64373.0,50.0,Female,18.0,19.0,7.0,22.0,Standard,Monthly,Bin3,13.0,-0.187097,0,1,0,1.0


### 6. Define at least one new attribute based on existing attribute, and explain your reason behind your definition

Attribute purpose: Support request frequency <br>
---
Measured by: Support Calls / Tenure <br>
---
Explanation:<br>
<ul>
<li> By dividing the total support calls by the customer tenure, we can know how often a customer reach for assistance while using the service.
<li> The more often customer requests support, the more frustration they may experience, and this can be a good attribute to measure customer churn.
</ul>

In [42]:
df["Support request frequency"] = df["Support Calls"] / df["Tenure"]
df['Churn'], df['Support request frequency']  = df['Support request frequency'], df['Churn']
df.rename(columns={'Churn': 'Support request frequency', 'Support request frequency': 'Churn'}, 
                    inplace=True)

In [43]:
df.tail()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,LI z-score,Annual,Monthly,Quarterly,Support request frequency,Churn
64369,64370.0,45.0,Female,33.0,12.0,6.0,21.0,Basic,Quarterly,Bin5,14.0,-0.070929,0,0,1,0.181818,1.0
64370,64371.0,37.0,Male,6.0,1.0,5.0,22.0,Standard,Annual,Bin5,9.0,-0.651765,1,0,0,0.833333,1.0
64371,64372.0,25.0,Male,39.0,14.0,8.0,30.0,Premium,Monthly,Bin2,20.0,0.626073,0,1,0,0.205128,1.0
64372,64373.0,50.0,Female,18.0,19.0,7.0,22.0,Standard,Monthly,Bin3,13.0,-0.187097,0,1,0,0.388889,1.0
64373,64374.0,52.0,Female,45.0,15.0,9.0,25.0,Standard,Monthly,Bin4,22.0,0.858408,0,1,0,0.200000,1.0
